In [ ]:
import numpy as np
import mdptoolbox, mdptoolbox.example
import random
from numpy.random import seed
import time



#np.random.seed(0)
#P, R = mdptoolbox.example.forest()
s = 100
a= 10
discount = 0.9
episodes = 1 #100
iterations = 50
w_val = 1.3

vi_diff = np.zeros((episodes,iterations,1))
newton_diff= np.zeros((episodes,iterations,1))
gen_newton_diff = np.zeros((episodes,iterations,1))

# vi_pol_diff = np.zeros((episodes,iterations,1))
# newton_pol_diff = np.zeros((episodes,iterations,1))

vi_time = np.zeros((episodes,1))
newton_time = np.zeros((episodes,1))

for count in range(episodes):
    
    np.random.seed((count+1)*100)
    random.seed((count+1)*110)
   
    P, R = mdptoolbox.example.rand(s, a)

    vi = mdptoolbox.mdp.ValueIteration(P, R, discount,epsilon=0.0000001,max_iter = iterations)
    vi.run()
    
    start = time.time()
    vi3 = mdptoolbox.mdp.QValueIteration(P,R,discount,max_iter = iterations,opt = vi.V, opt_pol = vi.policy)
    end = time.time()
    
    vi3.run()
    
    vi_time[count] = end-start

    
    start = time.time()
    vi4 = mdptoolbox.mdp.NewtonQValueIteration(P,R,discount,max_iter = iterations,opt = vi.V, opt_pol = vi.policy)
    end = time.time()
    
    vi4.run()
    
    newton_time[count] = end-start
    
    vi5 = mdptoolbox.mdp.GenNewtonQValueIteration(P,R,discount,max_iter = iterations,opt = vi.V, opt_pol = vi.policy,w = w_val)
    end = time.time()
    
    vi5.run()
    
    
    vi_diff[count] = vi3.store_error
    newton_diff[count] = vi4.store_error
    gen_newton_diff[count] = vi5.store_error
    

print(np.linalg.norm(vi.V - np.max(vi3.Q,axis = 0),ord=np.inf)) #Q-bellman
print(np.linalg.norm(vi.V - np.max(vi4.nmodQ,axis = 0),axis=0,ord=np.inf))
print(np.linalg.norm(vi.V - np.max(vi5.nmodQ,axis = 0),axis=0,ord=np.inf))